In [2]:
from bs4 import BeautifulSoup # For web scraping
import requests               #allows us to make use of HTTP within our Python programs in a human readable way
import html5lib
import csv

In [119]:
url = 'https://www.imdb.com/chart/top?ref_=nv_mv_250'   #url to Top rated movies
r = requests.get(url)                                   #r = request object
soup = BeautifulSoup(r.content, 'html5lib')             #html5lib = html parser | soup = BeautifulSoup object
tables = soup.find_all('td', class_ = 'titleColumn')    
 

In [120]:
#Tried to put dictionary data into the list 

IMDBList = []                  #List defined

for t in tables:
    IMDBDic = {}
    u = t.a['href']
    url1 = 'https://www.imdb.com' + u              #created a link
    
    r1 = requests.get(url1)
    
    soup1 = BeautifulSoup(r1.content, 'html5lib')
    
    IMDBDic['Link'] = url1.encode('utf-8')    #Encode- To replace unicode into str. 
    
    IMDBDic['NameOfTheMovie'] = (soup1.find('div', class_='title_wrapper').h1.text).strip().encode('utf-8')
    
    IMDBDic['YearRelease'] = (soup1.find('span', id="titleYear").a.text).strip().encode('utf-8')
    
    IMDBDic['IMDBratingValue'] = (soup1.find('span', itemprop="ratingValue").text).strip().encode('utf-8')
    
    IMDBDic['NoOfReviewer'] = (soup1.find('span', itemprop="ratingCount").text).strip().encode('utf-8')
    
    IMDBDic['LengthOfTheMovie'] = (soup1.find('div', class_='subtext').time.text).strip().encode('utf-8')
    
    IMDBDic['StorySummary'] = (soup1.find('div', class_="summary_text").text).strip().encode('utf-8')
    
    
    WrDrAc = soup1.find_all('div', class_="credit_summary_item")    ##Pulling Writer, Director and Actors data

    w=[]
    a=[]

    for Wr in WrDrAc:                         

        if Wr.h4.text == 'Director:':
            Director = Wr.a.text
    
        if Wr.h4.text == 'Writers:':
            Writers = Wr.find_all('a')
            for Writer in Writers:
                w1 = Writer.contents[0]
                w.append(w1)
    
        for i in range(len(w)):
            if i == 1:
                IMDBDic['Writer1'] = w[0].strip().encode('utf-8')
            if i == 2:
                IMDBDic['Writer2'] = w[1].strip().encode('utf-8')
            if i == 3:
                IMDBDic['Writer3'] = w[2].strip().encode('utf-8')
                
        
        if Wr.h4.text == 'Stars:':
            Stars = Wr.find_all('a')
            for Star in Stars:
                s1 = Star.contents[0]
                a.append(s1)
    
        for i in range(len(a)):
            if i == 1:
                IMDBDic['Star1'] = a[0].strip().encode('utf-8')
            if i == 2:
                IMDBDic['Star2'] = a[1].strip().encode('utf-8')
            if i == 3:
                IMDBDic['Star3'] = a[2].strip().encode('utf-8')
            if i == 4:
                IMDBDic['Star4'] = a[3].strip().encode('utf-8')
            if i == 5:
                IMDBDic['Star5'] = a[4].strip().encode('utf-8')


    IMDBDic['DirectorName'] = Director.strip().encode('utf-8')
    
    
    KeyW_Genres = soup1.find_all('div', class_="see-more inline canwrap")   #Pulling Genres and keywords data

    KeyWords = ""
    Genres = []

    for KeyW_Genre in KeyW_Genres:
        if KeyW_Genre.h4.text == 'Plot Keywords:':
            KeyWs = KeyW_Genre.find_all('span')
            for KeyW in KeyWs:
                Key = KeyW.contents[0]
                KeyWords += Key
            
        if KeyW_Genre.h4.text == 'Genres:':
            Gens = KeyW_Genre.find_all('a')
            for Gen in Gens:
                Genre = Gen.contents[0]
                Genres.append(Genre)
            
        for i in range(len(Genres)):
            if i == 0:
                IMDBDic['Genre1'] = Genres[0].strip().encode('utf-8')
            if i == 1:
                IMDBDic['Genre2'] = Genres[1].strip().encode('utf-8')
            if i == 2:
                IMDBDic['Genre3'] = Genres[2].strip().encode('utf-8')
            if i == 3:
                IMDBDic['Genre4'] = Genres[3].strip().encode('utf-8')

            
    IMDBDic['PlotKeywordsList'] = KeyWords.strip().encode('utf-8')

    
    IMDBList.append(IMDBDic)                                         #appending dictionary into list


In [121]:
filename = 'C:\Users\Mangesh\Desktop\IMDBRating.csv'
with open(filename, 'w') as f: 
    w = csv.DictWriter(f,['NameOfTheMovie','Link','YearRelease','IMDBratingValue','NoOfReviewer','LengthOfTheMovie','Genre1','Genre2','Genre3','Genre4','StorySummary','DirectorName','Writer1','Writer2','Writer3','Star1','Star2','Star3','Star4','Star5','PlotKeywordsList']) 
    w.writeheader()
    for IMDBDic in IMDBList: 
        w.writerow(IMDBDic)  